In [ ]:
import tifffile
import torch
from pathlib import Path 
import plotly.express as px
import numpy as np

ROOT_DIR = Path("../")

In [ ]:
mask = tifffile.imread(ROOT_DIR / "train/a_GT/man_seg0474.tif") != 0

mask_gradients = tifffile.imread(ROOT_DIR / "train/a_flow/seg0474_gradients.tif")
mask_gradients = torch.tensor(mask_gradients[[1, 0], :, :]).unsqueeze(0)

batch_size = mask_gradients.shape[0]
image_height = mask_gradients.shape[2]
image_width = mask_gradients.shape[3]

# positions in (batch_size, image_height, image_width, 2)
positions = (
    torch.cartesian_prod(
        torch.arange(image_height),
        torch.arange(image_width),
    )
    .reshape(image_height, image_width, 2)
    .unsqueeze(0)
    .repeat(batch_size, 1, 1, 1)
).double()

In [ ]:
mask.shape

In [ ]:
positions /= (
    torch.tensor([image_height, image_width]).double().unsqueeze(0).unsqueeze(0)
)
positions = positions * 2 - 1

mask_gradients *= 2.0 / torch.tensor([image_height, image_width]).unsqueeze(0).unsqueeze(-1).unsqueeze(-1)

# positions = positions[:, :, :, [1, 0]]
for _ in range(100):
    gradients = torch.nn.functional.grid_sample(
        mask_gradients, positions, align_corners=False
    )
    for k in range(2):
        positions[:,:,:,k] = torch.clamp(positions[:,:,:,k] + gradients[:,k,:,:], -1.0, 1.0)

In [ ]:
positions = (positions + 1) / 2
positions *= (
    torch.tensor([image_height, image_width]).double().unsqueeze(0).unsqueeze(0)
)
px.scatter(
    x=positions[0, :, :, 1][mask], y=positions[0, :, :, 0][mask]
).update_yaxes(scaleanchor="x", scaleratio=1,).update_layout(width=800, height=800)